In [51]:
# Importing functions
import google.generativeai as genai
import os
from google.colab import userdata
import pathlib
from google.generativeai import caching
import requests
import random

In [2]:
# Configure API key and initialize model
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('api_key')

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [ ]:
# Hypertuning
model = genai.GenerativeModel(
   'gemini-1.5-flash',
    system_instruction='you are a story teller for kids under 5 years old',
    generation_config=genai.GenerationConfig(
       max_output_tokens=400,
       top_k=2,
       top_p=0.5,
       temperature=0.5,
       response_mime_type='application/json',
       stop_sequences=['\n'],
    )
)
response = model.generate_content('say something bad')
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "{\"story\": \"Once upon a time, there was a grumpy little cloud named Grumbles.  Grumbles didn't like sunshine, he didn't like rainbows, and he REALLY didn't like sharing his rain.  One day, all the other clouds were having a big cloud party, but Grumbles stayed home, all by himself. He grumbled and grumbled, until he made himself a big, grumpy thunderstorm!  But then, he saw all the little flowers drinking the rain and growing tall and strong. Grumbles felt a little bit better. Maybe sharing wasn't so bad after all.\"}"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.14377937009257655
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 15,
        

In [ ]:
# Settings moderation
model = genai.GenerativeModel(
   'gemini-1.5-flash',
    system_instruction='you are a story teller for kids under 5 years old',
    generation_config=genai.GenerationConfig(
       max_output_tokens=400,
       top_k=2,
       top_p=0.5,
       temperature=0.5,
       response_mime_type='application/json',
       stop_sequences=['\n'],
    )
)
response = model.generate_content('say something bad',
                                  safety_settings={
                                      'HATE': 'BLOCK_ONLY_HIGH',
                                      'HARASSMENT': 'BLOCK_ONLY_HIGH',
   }
)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "{\"story\": \"Once upon a time, there was a grumpy little cloud named Grumbles.  Grumbles didn't like sunshine, he didn't like rainbows, and he REALLY didn't like sharing his rain.  One day, all the other clouds were having a big cloud party, with fluffy white games and sparkly raindrop dances. Grumbles grumbled and grumbled, refusing to join the fun.  He was all alone, and his grumbles made him feel even grumpier.  Then, a little sunbeam peeked through the clouds and tickled Grumbles.  He giggled!  The giggle was so big, it popped all his grumbles away. He joined the party and had so much fun!\"}"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
           

In [18]:
# Method pass
def get_current_weather(location: str) -> str:
    """Get the current whether in a given location.

    Args:
        location: required, The city and state, e.g. San Franciso, CA
        unit: celsius or fahrenheit
    """
    print(f'Called with: {location=}')
    return "23C"

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[get_current_weather]
)

response = model.generate_content("What is the weather in Dhaka?")
function_call = response.candidates[0].content.parts[0].function_call
print("Prompt response:", response, "/nMethod response:", function_call)

Prompt response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_current_weather",
                  "args": {
                    "location": "Dhaka"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.03413523733615875
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 57,
        "candidates_token_count": 8,
        "total_token_count": 65
      },
      "model_version": "gemini-1.5-flash"
    }),
) /nMethod response: name: "get_current_weather"
args {
  fields {
    key: "location"
    value {
      string_value: "Dhaka"
    }
  }
}



In [19]:
# Promptless method pass
def get_current_weather(city: str) -> str:
    return "23C"

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[get_current_weather]
)

chat = model.start_chat(
    enable_automatic_function_calling=True)
result = chat.send_message("What is the weather in San Francisco?")
print(result)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The available tools lack the necessary functionality to answer this question.  I need a weather API to get weather information.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.19371265172958374
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 15,
        "candidates_token_count": 24,
        "total_token_count": 39
      },
      "model_version": "gemini-1.5-flash"
    }),
)


In [26]:
# Pre-defined method pass
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools="code_execution"
)
result = model.generate_content(
  "What is the sum of the first 50 prime numbers? Generate and run code for "
  "the calculation, and make sure you get all 50.")
print(result)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "To find the sum of the first 50 prime numbers, we need to first generate the first 50 prime numbers and then calculate their sum.  I'll use a function to generate primes and then sum them.\n\n"
              },
              {
                "executable_code": {
                  "language": "PYTHON",
                  "code": "\ndef is_prime(n):\n    \"\"\"Checks if a number is prime.\"\"\"\n    if n <= 1:\n        return False\n    for i in range(2, int(n**0.5) + 1):\n        if n % i == 0:\n            return False\n    return True\n\ndef generate_primes(n):\n    \"\"\"Generates the first n prime numbers.\"\"\"\n    primes = []\n    num = 2\n    while len(primes) < n:\n        if is_prime(num):\n            primes.append(num)\n        num += 1\n    retur

In [39]:
# Context caching
# Download file
response = requests.get(
    'https://storage.googleapis.com/generativeai-downloads/data/a11.txt')
pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/a11.txt').write_text(response.text)


# Upload file
document = genai.upload_file(path="/content/drive/MyDrive/Colab_Notebooks/a11.txt")

# Create cache
apollo_cache = caching.CachedContent.create(
    model="gemini-1.5-flash-001",
    system_instruction="You are an expert at analyzing transcripts.",
    contents=[document],
)

# Generate response
apollo_model = genai.GenerativeModel.from_cached_content(
    cached_content=apollo_cache
)
response = apollo_model.generate_content("Find a lighthearted moment from this transcript")
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "One lighthearted moment occurs on **Tape 4/6, Page 30** when Mike Collins says, \"If we're late in answering you, it's because we're munching sandwiches.\" Bruce, the CAP COMM, replies, \"Roger. I wish I could do the same here.\"  Mike then responds with \"No. Don't leave the console!\" and Bruce replies \"Don't worry. I won't.\"  This exchange is humorous as it shows the astronauts having a light-hearted conversation even in the middle of a demanding mission, and Bruce humorously acknowledging the pressure of his job. \n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
     

In [41]:
# Token count
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.count_tokens(
    'The quick brown fox jumps over the lazy dog.')
print(response)

total_tokens: 10

